In [1]:
import numpy as np
import torch 
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.tensorboard import SummaryWriter
import torchvision.transforms as transforms
import torchvision.models as models
import os
import joblib
import glob
from skimage import io
import random

In [2]:
class MyDataset(data.Dataset):
    def __init__(self, mode, transform, val_size=-1):
        super().__init__()
        self.transform = transform
        self.mode = mode
        if mode == 'train':
            all_paths = glob.glob('/kaggle/input/intel-image-classification/seg_train/seg_train/**/*.jpg', recursive=True)
            random.Random(0).shuffle(all_paths)
            self.images_paths = all_paths[0: int(len(all_paths) * (1 - val_size))]
        elif mode == 'val':
            all_paths = glob.glob('/kaggle/input/intel-image-classification/seg_train/seg_train/**/*.jpg', recursive=True)
            random.Random(0).shuffle(all_paths)
            self.images_paths = all_paths[int(len(all_paths) * (1 - val_size)): ]
        elif mode == 'test':
            self.images_paths = glob.glob('/kaggle/input/intel-image-classification/seg_test/seg_test/**/*.jpg', recursive=True)
        self.labels_to_int = {'buildings': 0, 'forest': 1, 'glacier': 2, 'mountain': 3, 'sea': 4, 'street': 5}

    def __getitem__(self, index):
        img = io.imread(self.images_paths[index])
        img = self.transform(img)
        label = self.images_paths[index].split(os.sep)[-2]
        label = self.labels_to_int[label]
        return img, label

    def __len__(self):
        return len(self.images_paths)

In [3]:
class ResNext(nn.Module):
    def __init__(self):
        super().__init__()
        self.resnext = models.resnext50_32x4d(pretrained=True)
        self.fc = nn.Linear(1000, 6)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.resnext(x)
        x = self.relu(x)
        x = self.fc(x)
        return x
    
class DenseNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.densenet = models.densenet201(pretrained=True)
        self.fc = nn.Linear(1000, 6)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.densenet(x)
        x = self.relu(x)
        x = self.fc(x)
        return x

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding='same')
        self.bn1 = nn.BatchNorm2d(16)
        self.max_pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding='same')
        self.bn2 = nn.BatchNorm2d(32)
        self.max_pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(32, 64, 3, padding='same')
        self.bn3 = nn.BatchNorm2d(64)
        self.max_pool3 = nn.MaxPool2d(2)
        self.conv4 = nn.Conv2d(64, 128, 3, padding='same')
        self.bn4 = nn.BatchNorm2d(128)
        self.max_pool4 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(10368, 50)
        self.fc2 = nn.Linear(50, 6)
        self.relu = nn.ReLU()


    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.max_pool2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.max_pool3(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu(x)
        x = self.max_pool4(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [4]:
class Accuracy(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, pred, y_true):
        return (torch.argmax(pred, axis=1) == y_true).sum() / y_true.shape[0]

In [5]:
class Trainer:
    def __init__(self, model, learning_rate, batch_size, loss_func, metric, num_epochs, decay, decay_every, train_dataset, val_dataset, test_dataset, tensorboard_dir):
        self.train_dataset = train_dataset
        self.val_dataset = val_dataset
        self.test_dataset = test_dataset
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.model = model.to(self.device)
        print(self.model)
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.loss_func = loss_func
        self.metric = metric
        self.num_epochs = num_epochs
        self.decay = decay
        self.decay_every = decay_every
        self.writer = SummaryWriter(tensorboard_dir)
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate)
        self.train_loader = data.DataLoader(dataset=self.train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=2, pin_memory=True)
        self.val_loader = data.DataLoader(dataset=self.val_dataset, batch_size=self.batch_size, shuffle=False, num_workers=2, pin_memory=True)
        self.test_loader = data.DataLoader(dataset=self.test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=2, pin_memory=True)
        self.scheduler = StepLR(self.optimizer, self.decay_every, gamma=self.decay)

    def fit(self):
        self.fit_all_epochs()
        self.writer.flush()
        self.writer.close()

    def fit_all_epochs(self):
        for epoch in range(1, self.num_epochs + 1):
            print('epoch:', epoch)
            self.fit_epoch()   
            train_loss, train_metric = self.evaluate('train')  
            val_loss, val_metric = self.evaluate('val') 
            test_loss, test_metric = self.evaluate('test') 
            title = f'lr={self.learning_rate}, bs={self.batch_size}, decay={self.decay}, decay_every={self.decay_every}'
            self.writer.add_scalars('loss: ' + title, {'train loss': train_loss,
                                                       'validation loss': val_loss,
                                                       'test loss':test_loss}, epoch)
            self.writer.add_scalars('metric: ' + title, {'train metric': train_metric,
                                                         'validation metric': val_metric,
                                                         'test metric':test_metric}, epoch)
            self.scheduler.step()


    def fit_epoch(self):
        self.model.train()
        epoch_loss = 0
        for i, (samples, targets) in enumerate(self.train_loader):
            samples = samples.to(self.device).float()
            targets = targets.to(self.device).long()
            preds = self.model(samples)
            loss = self.loss_func(preds, targets)
            #print('iteration:', i, 'loss:', loss.item())
            epoch_loss += loss.item()
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()


    def evaluate(self, stage):
        self.model.eval()  
        if stage == 'val':
            loader = self.val_loader
        elif stage == 'test':
            loader = self.test_loader
        elif stage == 'train':
            loader = self.train_loader
        loss, metric = 0, 0
        with torch.no_grad():
            for samples, targets in loader:
                samples = samples.to(self.device)
                targets = targets.to(self.device) 
                preds = self.model(samples)
                loss += self.loss_func(preds, targets).item()
                metric += self.metric(preds, targets).item()


            loss = loss / len(loader)
            metric = metric / len(loader)
            if stage == 'train':
                print('train -', 'loss:', f'{loss:.2}', 'metric:', f'{metric:.2}')
            elif stage == 'val':
                print('val -', 'loss:', f'{loss:.2}', 'metric:', f'{metric:.2}')
            elif stage == 'test':
                print('test -', 'loss:', f'{loss:.2}', 'metric:', f'{metric:.2}')
        
        return loss, metric

In [6]:
val_size = 0.1
loss_func = nn.CrossEntropyLoss()
metric = Accuracy()
batch_size = 32
learning_rate = 0.001
num_epochs = 20
decay = 0.1
decay_every = 7
tensorboard_dir = 'run'

train_transform = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((150, 150)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.GaussianBlur(5),
                                      transforms.RandomAdjustSharpness(2),
                                      transforms.RandomAutocontrast(),
                                      transforms.ColorJitter(brightness=0.5, hue=0.3)])
test_transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((150, 150))])

#fit
train_dataset = MyDataset('train', train_transform, val_size)
val_dataset = MyDataset('val', test_transform, val_size)
test_dataset = MyDataset('test', test_transform)
resnext_model = ResNext()
os.makedirs(tensorboard_dir, exist_ok=True)
resnext_trainer = Trainer(resnext_model, learning_rate, batch_size, loss_func, metric, num_epochs,
                  decay, decay_every, train_dataset, val_dataset, test_dataset, tensorboard_dir)
resnext_trainer.fit()

#save
torch.save(resnext_model.state_dict(), 'resnext_model.ckpt')

Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth


  0%|          | 0.00/95.8M [00:00<?, ?B/s]

ResNext(
  (resnext): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Seque

In [ ]:
val_size = 0.1
loss_func = nn.CrossEntropyLoss()
metric = Accuracy()
batch_size = 32
learning_rate = 0.001
num_epochs = 20
decay = 0.1
decay_every = 7
tensorboard_dir = 'run'

train_transform = transforms.Compose([transforms.ToTensor(),
                                      transforms.Resize((150, 150)),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.GaussianBlur(5),
                                      transforms.RandomAdjustSharpness(2),
                                      transforms.RandomAutocontrast(),
                                      transforms.ColorJitter(brightness=0.5, hue=0.3)])
test_transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((150, 150))])

#fit
train_dataset = MyDataset('train', train_transform, val_size)
val_dataset = MyDataset('val', test_transform, val_size)
test_dataset = MyDataset('test', test_transform)
densenet_model = DenseNet()
os.makedirs(tensorboard_dir, exist_ok=True)
densenet_trainer = Trainer(densenet_model, learning_rate, batch_size, loss_func, metric, num_epochs,
                  decay, decay_every, train_dataset, val_dataset, test_dataset, tensorboard_dir)
densenet_trainer.fit()

#save
torch.save(densenet_model.state_dict(), 'densenet_model.ckpt')

Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/hub/checkpoints/densenet201-c1103571.pth


  0%|          | 0.00/77.4M [00:00<?, ?B/s]

DenseNet(
  (densenet): DenseNet(
    (features): Sequential(
      (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu1): ReLU(inplace=True)
          (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu2): ReLU(inplace=True)
          (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (denselayer2): _DenseLayer(
          (norm1): BatchNorm2d(96, eps=1e-05, momentum=0

KeyboardInterrupt: 